<a href="https://colab.research.google.com/github/jantic/DeOldify/blob/master/ImageColorizerColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#◢ DeOldify - Colorize Your Own Photos!

##Use this Colab notebook to colorize black & white photos in four simple steps.
1. Specify photo URL- make sure it's a direct link to the photo (with extension of .jpg, .png, etc).
2. Select 'Render Factor'.  Generally, older and lower quality photos will render bettter with lower render factors (14-21 range) while higher quality photos will do better on higher render factors.
3. Colorize your photo with DeOldify
4. Save a copy to your device by right clicking on the rendered image and selecting save.

---

####**Credits:**

Special thanks to:

Matt Robinson and María Benavente for pioneering the DeOldify image colab notebook.  

Dana Kelley for doing things, breaking stuff & having an opinion on everything.



---


#◢ Verify Correct Runtime Settings

**<font color='#FF000'> IMPORTANT </font>**

In the "Runtime" menu for the notebook window, select "Change runtime type." Ensure that the following are selected:
* Runtime Type = Python 3
* Hardware Accelerator = GPU 


In [ ]:
from os import path
import torch

#◢ Git clone and install DeOldify

In [ ]:
!git clone -b FastAIv1 --single-branch https://github.com/jantic/DeOldify.git DeOldify
#!git clone https://github.com/jantic/DeOldify.git DeOldify

In [ ]:
cd DeOldify

#◢ Setup

In [ ]:
!pip install PyDrive
!pip install ffmpeg-python
!pip install youtube-dl
!pip install tensorboardX

In [ ]:
import fastai
from fasterai.visualize import *
from pathlib import Path
torch.backends.cudnn.benchmark=True

In [ ]:
!mkdir 'models'
!wget wget https://www.dropbox.com/s/zkehq1uwahhbc2o/ColorizeArtistic_gen.pth?dl=0 -O ./models/ColorizeArtistic_gen.pth

In [ ]:
colorizer = get_image_colorizer(artistic=True)

#◢ Image URL

Any direct link to an image should do (will end with extension .jpg, .png, etc).  NOTE: If you want to use your own image, upload it first to a site like Imgur. 

In [ ]:
source_url = '' #@param {type:"string"}

#◢ Render Factor

The default value of 35 has been carefully chosen and should work -ok- for most scenarios (but probably won't be the -best-). This determines resolution at which the photo is rendered. Lower resolution will render faster, and colors also tend to look more vibrant.  Older and lower quality photos in particular will generally benefit by lowering the render factor. Higher render factors are often better for higher quality photos.

In [ ]:
render_factor = 35  #@param {type: "slider", min: 7, max: 46}

#◢ Run DeOldify

In [ ]:
if source_url is not None and source_url !='':
    colorizer.plot_transformed_image_from_url(path="test_images/image.jpg", url=source_url, render_factor=render_factor)
else:
    print('Provide an image url and try again.')

#◢ Download

* In the menu to the left, click **Files**
* If you don't see the 'DeOldify' folder, click "Refresh"
* By default, rendered image will be in /DeOldify/result_images/

---
#⚙ Recommended image sources 
* [/r/TheWayWeWere](https://www.reddit.com/r/TheWayWeWere/)